Import required packages

In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from copy import deepcopy
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

Read the file

In [4]:
file = pd.read_excel('shortcourses2566.xlsx')

Count the number of courses

In [5]:
course_counts = pd.DataFrame(file)['หลักสูตรอบรมระยะสั้น'].value_counts()
courses = pd.Series(course_counts.index)
courses = courses.sort_values().set_axis(range(0,len(courses)))

Create Series of users

In [6]:
s_name = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
Users = pd.Series(s_name, name='User')
Users

0             PORPHAING JANTIP
1         THANAKORN DARASRISAK
2             KARNJANA EAMTANG
3             KARNJANA EAMTANG
4       THITIMUN VORATHONGCHAI
                 ...          
6122         KITTINON SANTASUP
6123      WITTAWAT SERMSRIPONG
6124               NON NAKKARA
6125         PHATTHARAPHON LIN
6126      THANAPAT SEEMAKAJOHN
Name: User, Length: 6127, dtype: object

Create Series of emails

In [7]:
s_email = file.loc[:, 'อีเมล'].fillna("")
Emails = pd.Series(s_email ,name='Email')
Emails

0            tanghaoren17@gamil.com
1               tuakung21@gmail.com
2            karnjana.aon@gmail.com
3            karnjana.aon@gmail.com
4          thitimun.rama1@gmail.com
                   ...             
6122           kittinon_s@cmu.ac.th
6123        wittawat_serm@cmu.ac.th
6124          non_nakkara@cmu.ac.th
6125    phattharaphon_lin@cmu.ac.th
6126         thanapat_see@cmu.ac.th
Name: Email, Length: 6127, dtype: object

Provide a score to each user based on their email domain

In [8]:
email_score = []
for data in Emails:
    if data != '':
        if data.split('@')[1] == 'cmu.ac.th':
            email_score.append(2)
        else:
            email_score.append(1)
    else:
        email_score.append(0)
email_score = pd.Series(email_score, name='Score Email')

email_score

0       1
1       1
2       1
3       1
4       1
       ..
6122    2
6123    2
6124    2
6125    2
6126    2
Name: Score Email, Length: 6127, dtype: int64

Email Score Statistic

In [9]:
zero_score_count = email_score.where(email_score == 0).count()
one_score_count = email_score.where(email_score == 1).count()
two_score_count = email_score.where(email_score == 2).count()

print("Number of students who filled cmu email:", two_score_count)
print("Number of students who filled other email:", one_score_count)
print("Number of students who did not fill email:", zero_score_count)

Number of students who filled cmu email: 1268
Number of students who filled other email: 4853
Number of students who did not fill email: 6


Create function to calculate age-education score

In [10]:
def getAgeEducationScore(age, limit_age):
    if age <= limit_age:
        score = 0
    elif age <= limit_age + 2:
        score = 1
    else:
        score = 2
    return score

Create set of the educational range

In [11]:
set_nan = {'อื่นๆ (-)', np.nan}
set_primaryschool = {'ประถมศึกษา', 'อื่นๆ (ป.4)', 'อื่นๆ (ป.7)', 'อื่นๆ (ป7)'}
set_middleschool = {'มัธยมศึกษาตอนต้น', 'Secondary school', 'อื่นๆ (มศ.3)'}
set_highschool = {'มัธยมศึกษาตอนปลาย', 'High school', 'Vocational', 'การศึกษานอกระบบ', 
                  'ประกาศนียบัตรวิชาชีพ (ปวช.)', 'อื่นๆ (ม.ปลาย จบหลักสูตรEMR เป็นจนท.ปฏิบัติการ)',
                  'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาตอนปลาย)', 'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาปีที่6)', 
                  'อื่นๆ (มศ.5)'}
set_bachelor = {'ปริญญาตรี', 'Bachelor degree', 'Diploma', 'High Vocational', 
                'ประกาศนียบัตรวิชาชีพชั้นสูง (ปวส.)', 'อื่นๆ (กำลังศึกษาในระดับปริญญาตรี)', 
                'อื่นๆ (กำลังศึกษาปริญญาตรี สาขารังสีเทคนิค)', 'อื่นๆ (ปริญญาแพทยศาสตร์บัณฑิต)', 
                'อื่นๆ (นักศึกษาแพทย์ปี 5)', 'อื่นๆ (นักศึกษาแพทย์ มช ปี4 ศูนย์เชียงราย)', 
                'อื่นๆ (แพทยศาสตร์บัณฑิต)', 'อื่นๆ (แพทย์)', 'อื่นๆ (ประกาศณียบัตรผู้ช่วยพยาบาล)', 
                'อนุปริญญา', 'อื่นๆ (ป.ตรี)', 'อื่นๆ (ผู้ช่วยพยาบาล)'}
set_masterdocter = {'ปริญญาโท', 'ปริญญาเอก', "Master's degree", 'Other (OBGYN specalist lavel 1)', 
                    'Other (Residency)', 'Ph.D.', 'อื่นๆ (Internal Medicine)', 
                    'อื่นๆ (เฉพาะทาง)', 'อื่นๆ (วุฒิบัตร)', 'อื่นๆ (วว.ออร์โธปิดิกส์)', 
                    'อื่นๆ (วุฒิบัตรแสดงความรู้ความชำนาญในการประกอบวิชาชีพเภสัชกรรม สาขาเภสัชบำบัด)', 
                    'อื่นๆ (วุฒิบัตรผู้เชี่ยวชาญสาขาทันตกรรมทั่วไป)', 'อื่นๆ (วุฒิบัตรศัลยศาสตร์และแม็กซิลโลเฟเชียล)'}

list_degree = ((set_nan, 0), (set_primaryschool, 16), (set_middleschool, 19), 
               (set_highschool, 22), (set_bachelor,26), (set_masterdocter,40))

Create Series of Age-Education

In [12]:
ages = file.loc[:, 'อายุ']
educations = file.loc[:, 'วุฒิการศึกษา']
age_education_scores = []

for i,x in enumerate(educations):
    for y in list_degree:
        if x in y[0]:
            age_education_scores.append(getAgeEducationScore(ages[i], y[1]))
            
age_education_scores = pd.Series(age_education_scores, name='Age Education Score')

Age-Education Score Statistic

In [13]:
zero_score_count = age_education_scores.where(age_education_scores == 0).count()
one_score_count = age_education_scores.where(age_education_scores == 1).count()
two_score_count = age_education_scores.where(age_education_scores == 2).count()

print("Number of students who are currently in the educational system:", zero_score_count)
print("Number of students who were recently graduated:", one_score_count)
print("Number of students who are not in the educational system:", two_score_count)

Number of students who are currently in the educational system: 4229
Number of students who were recently graduated: 487
Number of students who are not in the educational system: 1411


Create Series of status

In [14]:
status = file.loc[:, 'สถานะ'].fillna("")
status = pd.Series(status ,name='Status')
status

0       ชำระเงิน
1       ชำระเงิน
2       ชำระเงิน
3       ชำระเงิน
4       ชำระเงิน
          ...   
6122    ชำระเงิน
6123    ชำระเงิน
6124    ชำระเงิน
6125    ชำระเงิน
6126    ชำระเงิน
Name: Status, Length: 6127, dtype: object

Provide a score to each user based on their purchase status

In [15]:
status_score = []
for x in status:
    if x == 'ชำระเงิน':
        status_score.append(8)
    if x == 'ไม่ผ่านการอนุมัติ':
        status_score.append(5)
    if x == 'ค้างชำระ':
        status_score.append(0)
status_score = pd.Series(status_score)

Purchase Status Score Statistics

In [16]:
zero_score_count = status_score.where(status_score == 0).count()
five_score_count = status_score.where(status_score == 5).count()
eight_score_count = status_score.where(status_score == 8).count()

print("Number of students who are in arrears:", zero_score_count)
print("Number of students whose payment was not approved:", five_score_count)
print("Number of students with payment approval:", eight_score_count)

Number of students who are in arrears: 531
Number of students whose payment was not approved: 124
Number of students with payment approval: 5472


Create Series of address

In [17]:
address = file.loc[:, 'ที่อยู่'].fillna("")
address = pd.Series(address ,name='Status')
address

0                                                        
1       125/27 ซ.3 ถ.ราษฎรยินดี ต.หน้าเมือง อ.เมือง จ....
2                                                        
3                                                        
4                                                        
                              ...                        
6122                                                     
6123                                                     
6124                                                     
6125                                                     
6126                                                     
Name: Status, Length: 6127, dtype: object

Provide a score to each user based on whether they provide address information or not

In [18]:
address_score = [ 0 if x == '' else 2 for x in address]
address_score = pd.Series(address_score)

Address Score Statistic

In [19]:
zero_score_count = address_score.where(address_score == 0).count()
two_score_count = address_score.where(address_score == 2).count()

print("Number of students who did not fill address:", zero_score_count)
print("Number of students who filled address:", two_score_count)

Number of students who did not fill address: 4005
Number of students who filled address: 2122


Create Series of data

In [20]:
enrollment = file.loc[:, 'วันที่สมัครอบรม'].fillna("")
enrollment = pd.Series(enrollment ,name='Date')
enrollment = enrollment.str.slice(start=-8, stop=-6)

Provide a score to each user based on their enrollment date

In [21]:
enrollment_set = {'08','09','10','15'}
enrollment_score = [ 1 if x in enrollment_set else 0 for x in enrollment]
enrollment_score = pd.Series(enrollment_score)

Create DataFrame by merging these 4 Series and calculate impressive level

In [22]:
user = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
course = file.loc[:, 'หลักสูตรอบรมระยะสั้น']
d = {
    'User': user,
    'Course': course,
    'Email Score': email_score,
    'Age Education Score': age_education_scores,
    'Enrollment': enrollment_score,
    'Payment Score': status_score,
    'Address Score': address_score,
    'Point': email_score + status_score + address_score + age_education_scores + enrollment_score,
    'Score': ( email_score + status_score + address_score + age_education_scores + enrollment_score ) / 15
}
df = pd.DataFrame(d)
df

,User,Course,Email Score,Age Education Score,Enrollment,Payment Score,Address Score,Point,Score
0,PORPHAING JANTIP,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,1,0,1,8,0,10,0.666667
1,THANAKORN DARASRISAK,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,1,1,1,8,2,13,0.866667
2,KARNJANA EAMTANG,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,1,2,0,8,0,11,0.733333
3,KARNJANA EAMTANG,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,1,2,0,8,0,11,0.733333
4,THITIMUN VORATHONGCHAI,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,1,2,0,8,0,11,0.733333
...,...,...,...,...,...,...,...,...,...
6122,KITTINON SANTASUP,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,2,0,0,8,0,10,0.666667
6123,WITTAWAT SERMSRIPONG,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,2,1,0,8,0,11,0.733333
6124,NON NAKKARA,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,2,0,0,8,0,10,0.666667
6125,PHATTHARAPHON LIN,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,2,0,0,8,0,10,0.666667


Create user-course table

In [23]:
# all user, course, score have the same length
data = {
    'User': df['User'],
    'Course': df['Course'],
    'Score': df['Score'],
}

predata = pd.DataFrame(data)
predata

,User,Course,Score
0,PORPHAING JANTIP,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.666667
1,THANAKORN DARASRISAK,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.866667
2,KARNJANA EAMTANG,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.733333
3,KARNJANA EAMTANG,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.733333
4,THITIMUN VORATHONGCHAI,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.733333
...,...,...,...
6122,KITTINON SANTASUP,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.666667
6123,WITTAWAT SERMSRIPONG,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.733333
6124,NON NAKKARA,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.666667
6125,PHATTHARAPHON LIN,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.666667


Calculate sparsity and csr matrix

In [24]:
# Pivot table by rotating course
data = pd.pivot_table(predata, values='Score', index='Course', columns='User').fillna(0)
data

User,AAIYANA MUSTAFA,ABDULFATTAH SENG,ABIGAIL LINE,ABRAM THANAWONG,ACHIRA TEAWSIRISUP,ACHIRAWIT AKSORNDIT,ACHIRAWIT PHUJOMCHAE,ACHIRAYA INNA,ACHIRAYA NUISEE,ACHIRAYA SUKPAISARN,...,YYY TTTT,ZABIDIN SENSANA,ZAR CHI LIN -,ZAYNUB NAEEM,ZE NYOI,ZHANGJINSHUO DONG,ZHENGHAO LIU,ZHENYI JIN,ZIWEN YONG,ZOE BESTMANNSMITH
Course,,,,,,,,,,,,,,,,,,,,,
Cancer Epidemiology and Prevention,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Coaching Skill : Crafting a New You สำหรับนักศึกษา มหาวิทยาลัยเชียงใหม่,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Coaching Skill : Crafting a New You สำหรับบุคลากร มหาวิทยาลัยเชียงใหม่,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Development Studies & Social Research Specialization 2023,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.733333,0.0,0.0,0.0,0.0,0.0
Digital Transformation: e-Document,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
เวชศาสตร์ครอบครัวขั้นสูง,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้นฟู มหาวิทยาลัยเชียงใหม่,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (Strengths Spotting : Turn on value and bring out the happiness) รุ่นที่ 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [25]:
# Item-User Rated Matrix
data_mtx = csr_matrix(data.values)

Euclidean Distance & Cosine Similarity

In [26]:
# model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=int(np.around(math.sqrt(len(courses))))).fit(data_mtx)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute').fit(data_mtx)
model_knn

NearestNeighbors(algorithm='brute', metric='cosine')

In [27]:
def recommender_knn(course_name, n_recommendations):
    idx = process.extractOne(course_name, courses)[2]
    print('Selected movie:', courses[idx], 'Index:', idx)
    distances, indices = model_knn.kneighbors(data_mtx[idx], n_neighbors=n_recommendations+1, return_distance=True)
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    course_distances = distances[0][1:]
    d = {
        'Course': recommended_courses,
        'Cosine Distance': course_distances
    }
    results = pd.DataFrame(data=d)
    return results

In [28]:
recommender_knn('หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (Basic Principle of Diagnostic Radiology Imaging Instruments)', 167)

Selected movie: หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (Basic Principle of Diagnostic Radiology Imaging Instruments) Index: 131


,Course,Cosine Distance
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.672148
42,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.722518
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.752072
163,เวชศาสตร์ครอบครัวขั้นสูง,0.932805
110,รังสีวิทยาวินิจฉัย,0.970761
...,...,...
62,การเขียนโครงการและกระบวนการใช้จ่ายเงินในกิจกรร...,1.000000
63,การเขียนโครงการและกระบวนการใช้จ่ายเงินในกิจกรร...,1.000000
64,การเขียนโครงการและกระบวนการใช้จ่ายเงินในกิจกรร...,1.000000
56,การเขียนโครงการและกระบวนการใช้จ่ายเงินในกิจกรร...,1.000000


Predata for hybrid recommendation

In [29]:
def recommender_knn_all_courses(course_name):
    model_knn.fit(data_mtx)
    idx = process.extractOne(course_name, courses)[2]
    # print('Selected movie:', courses[idx], 'Index:', idx)
    distances, indices = model_knn.kneighbors(data_mtx[idx], n_neighbors=len(courses))
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    scores = 1 - distances
    course_distances = scores[0][1:]
    d = {
        'Course': recommended_courses,
        'Score': course_distances
    }
    results = pd.DataFrame(data=d)
    results = results.sort_index().rename_axis('Index')
    return results

In [30]:
recommender_knn_all_courses('หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (Basic Principle of Diagnostic Radiology Imaging Instruments)')

,Course,Score
Index,,
0,Cancer Epidemiology and Prevention,0.000000
1,Coaching Skill : Crafting a New You สำหรับนักศ...,0.000000
2,Coaching Skill : Crafting a New You สำหรับบุคล...,0.000000
3,Development Studies & Social Research Speciali...,0.010863
4,Digital Transformation: e-Document,0.000000
...,...,...
163,เวชศาสตร์ครอบครัวขั้นสูง,0.067195
164,เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้น...,0.000000
165,แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (...,0.007425


In [31]:
def get_score_of_selected_courses(course_names):
    # Take index of the selected courses
    idx = [ process.extractOne(course_name, courses)[2] for course_name in course_names ]
    
    # take the selected course
    course_scores = []
    for course in courses:
        if course not in course_names:
            course_scores.append(recommender_knn_all_courses(course).iloc[idx])
            
    course_score_dict = {}
    for i,name in enumerate(course_names):
        course_score_dict[name] = []
        for score in course_scores:
            course_score_dict[name].append(score['Score'].iloc[i])
           
    course_score_df = pd.DataFrame(course_score_dict, index=range(0,len(course_scores)))
    
    return course_score_df.max()

In [32]:
def recommender_knn_by_user(user_name, n_recommendations):
    df = {
        'User': pd.Series(file['ชื่อ-นามสกุล (อังกฤษ)']),
        'Course': pd.Series(file['หลักสูตรอบรมระยะสั้น'])
    }
    
    user_course = pd.DataFrame(df)
    selected_user_name = user_course.loc[user_course['User'] == user_name]
    selected_courses = selected_user_name['Course']
    
    recommended_courses = [ recommender_knn_all_courses(x) for x in selected_courses]
    
    # pre dataframe
    df = pd.DataFrame({
        'Course': [],
        'Score': []
    }).rename_axis('Index')
    
    for x in recommended_courses:
        df = df._append(x)
    df =  df.sort_values('Score', ascending=False).drop_duplicates('Course')
    return df.head(n_recommendations)

Items show permanence whereas, people change with time
Items are fewer in numbers to deal with. Which leads to smaller similarity matrix. Amazon and Netflix use it!
Better for New users:
— Him selecting just one item will let us provide recommendations
— But for user based, new user has to wait until next build of similarity matrix (which is the only computational part of the framework)

In [33]:
recommender_knn_by_user('THANAKORN DARASRISAK', 10)

,Course,Score
Index,,
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.759024
131,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.277482
110,รังสีวิทยาวินิจฉัย,0.236569
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.178877
124,สารสนเทศทางสาธารณสุข 2566 (Public Health Infor...,0.079594
41,การวิจัยแบบผสมผสานทางสุขภาพ (Mixed Methods Res...,0.036606
136,หลักสูตรอบรมเข้มข้นระยะสั้นแพทย์-วิศวกรรมและวิ...,0.023389
98,ทักษะความเป็นนักดนตรี,0.022867
3,Development Studies & Social Research Speciali...,0.018290


Training and Testing

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

Predata of training

In [35]:
# Take 2 columns from Table
course_score = df[['Course', 'Score']]

# Sort the course
course = df['Course'].sort_values().unique()

# Calculate the mean of each course
course_mean = course_score.groupby('Course')
course_mean = course_mean.mean().loc[:, 'Score']

# Take 2 columns from Table
course_user = df[['Course', 'User']]

# Count the number of users who enrolled each course 
course_count = course_user.value_counts('Course')
course_count = course_count.sort_index()

# Take feature columns from Table
course_feature = df[['Course', 'Email Score', 'Age Education Score', 'Enrollment', 'Payment Score', 'Address Score']]
agg_functions = {
    'Email Score': 'mean',
    'Age Education Score': 'mean',
    'Enrollment': 'mean',
    'Payment Score': 'mean',
    'Address Score': 'mean',
}
course_feature = course_feature.groupby('Course').aggregate(agg_functions)
course_feature

,Email Score,Age Education Score,Enrollment,Payment Score,Address Score
Course,,,,,
Cancer Epidemiology and Prevention,1.000000,1.500000,0.500000,8.000000,0.000000
Coaching Skill : Crafting a New You สำหรับนักศึกษา มหาวิทยาลัยเชียงใหม่,1.714286,0.285714,0.107143,8.000000,2.000000
Coaching Skill : Crafting a New You สำหรับบุคลากร มหาวิทยาลัยเชียงใหม่,1.692308,1.410256,0.307692,6.358974,1.846154
Development Studies & Social Research Specialization 2023,1.028571,0.914286,0.200000,6.885714,0.400000
Digital Transformation: e-Document,1.863636,1.818182,0.681818,8.000000,0.545455
...,...,...,...,...,...
เวชศาสตร์ครอบครัวขั้นสูง,1.125000,1.000000,0.625000,8.000000,0.500000
เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้นฟู มหาวิทยาลัยเชียงใหม่,1.200000,0.800000,0.000000,8.000000,0.400000
แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (Strengths Spotting : Turn on value and bring out the happiness) รุ่นที่ 1,1.600000,1.100000,0.566667,8.000000,1.533333


Create the train dataframe

In [36]:
course_feature['Count'] = course_count
course_feature['Score'] = course_mean
train_df = pd.DataFrame(course_feature)
train_df

,Email Score,Age Education Score,Enrollment,Payment Score,Address Score,Count,Score
Course,,,,,,,
Cancer Epidemiology and Prevention,1.000000,1.500000,0.500000,8.000000,0.000000,2,0.733333
Coaching Skill : Crafting a New You สำหรับนักศึกษา มหาวิทยาลัยเชียงใหม่,1.714286,0.285714,0.107143,8.000000,2.000000,28,0.807143
Coaching Skill : Crafting a New You สำหรับบุคลากร มหาวิทยาลัยเชียงใหม่,1.692308,1.410256,0.307692,6.358974,1.846154,39,0.774359
Development Studies & Social Research Specialization 2023,1.028571,0.914286,0.200000,6.885714,0.400000,35,0.628571
Digital Transformation: e-Document,1.863636,1.818182,0.681818,8.000000,0.545455,22,0.860606
...,...,...,...,...,...,...,...
เวชศาสตร์ครอบครัวขั้นสูง,1.125000,1.000000,0.625000,8.000000,0.500000,8,0.750000
เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้นฟู มหาวิทยาลัยเชียงใหม่,1.200000,0.800000,0.000000,8.000000,0.400000,5,0.693333
แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (Strengths Spotting : Turn on value and bring out the happiness) รุ่นที่ 1,1.600000,1.100000,0.566667,8.000000,1.533333,30,0.853333


Split the train and the test

In [41]:
# Get X and y features variables
X = train_df[['Email Score', 'Age Education Score', 'Enrollment', 'Payment Score', 'Address Score', 'Count']]
y = train_df['Score']

# Use the train test split function
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=10, test_size=0.25
)

X

,Email Score,Age Education Score,Enrollment,Payment Score,Address Score,Count
Course,,,,,,
Cancer Epidemiology and Prevention,1.000000,1.500000,0.500000,8.000000,0.000000,2
Coaching Skill : Crafting a New You สำหรับนักศึกษา มหาวิทยาลัยเชียงใหม่,1.714286,0.285714,0.107143,8.000000,2.000000,28
Coaching Skill : Crafting a New You สำหรับบุคลากร มหาวิทยาลัยเชียงใหม่,1.692308,1.410256,0.307692,6.358974,1.846154,39
Development Studies & Social Research Specialization 2023,1.028571,0.914286,0.200000,6.885714,0.400000,35
Digital Transformation: e-Document,1.863636,1.818182,0.681818,8.000000,0.545455,22
...,...,...,...,...,...,...
เวชศาสตร์ครอบครัวขั้นสูง,1.125000,1.000000,0.625000,8.000000,0.500000,8
เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้นฟู มหาวิทยาลัยเชียงใหม่,1.200000,0.800000,0.000000,8.000000,0.400000,5
แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (Strengths Spotting : Turn on value and bring out the happiness) รุ่นที่ 1,1.600000,1.100000,0.566667,8.000000,1.533333,30


In [38]:
regressor = KNeighborsRegressor(n_neighbors=3)
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_test)
predictions

array([0.22751323, 0.68935863, 0.63839597, 0.67287638, 0.63182383,
       0.70740741, 0.65555556, 0.86111111, 0.70116402, 0.49363723,
       0.73185185, 0.52222222, 0.69539044, 0.78968254, 0.67407407,
       0.68844169, 0.85444444, 0.62222222, 0.53614164, 0.65509768,
       0.68668936, 0.68951639, 0.65185185, 0.69482657, 0.68055556,
       0.7262963 , 0.72075163, 0.65509768, 0.70515873, 0.63182383,
       0.65837633, 0.65902993, 0.60987197, 0.72329174, 0.68951639,
       0.6818104 , 0.40761905, 0.67756237, 0.22751323, 0.70750678,
       0.6       , 0.58827839])

In [39]:
print("Mean Squared Error:", mean_squared_error(predictions,y_test))
print("Mean Absolute Error:", mean_absolute_error(predictions,y_test))

NameError: name 'mean_squared_error' is not defined